## Local Setup

If you prefer to work locally, see the following instructions for setting up Python in a virtual environment. You can then ignore the instruction in "Colab Setup".

Tips:
- As this exercise could require substantial computation, we recommend using GPU for training. If you do not have GPU on local machine, please consider using Colab instead
- The D4RL and mujoco-py that we will use in this exercise are unfortunately quite tricky to install, please conisder to use Colab instead
- If you still want to run the notebook locally, here is an installation guide for reference. Notice: the setup instruction is tested with Ubuntu 22.04.

If you haven't yet, create a [conda](https://docs.conda.io/projects/conda/en/latest/user-guide/install/index.html) environment using:
```
conda create --name rl_hw5 python=3.8
conda activate rl_hw5
```

Install the required conda packages:
```
conda install -c conda-forge glew
conda install -c conda-forge mesalib
conda install -c anaconda mesa-libgl-cos6-x86_64  
conda install -c menpo glfw3  
```

Download [mujoco 2.1.0](https://github.com/google-deepmind/mujoco/releases/tag/2.1.0) and place it under your `$HOME/.mujoco/` , unzip the `tar.gz` file and rename the folder to `mujoco210`. The folder structure should look like following:
```
[user] cd mujoco210
[user] pwd
/home/(username)/.mujoco/mujoco210
[user] ls
bin  include  model  sample  THIRD_PARTY_NOTICES
```
Now we can try to install the [D4RL](https://github.com/Farama-Foundation/D4RL) module, you could either follow the instruction guidance from the official repository or the following guides:
```
pip install git+https://github.com/tinkoff-ai/d4rl@master#egg=d4rl

pip install patchelf
pip install 'cython<3'
conda env config vars set LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$HOME/.mujoco/mujoco210/bin:/usr/lib/nvidia
```
Reactivate the conda environment to make the environmental variable setting take effect:
```
conda deactivate
conda activate rl_hw5
```
Test if the d4rl works, open a python interactive console, run the following codes:
You can safely ignore several warnings, as long as you can import the module successfully.
```
import d4rl
import d4rl.gym_mujoco
import gym
env = gym.make("halfcheetah-medium-expert-v2")
```

Install Torch using conda **or** pip, so run e.g.:
```
conda install pytorch pytorch-cuda=11.8 -c pytorch -c nvidia
```
**or**
```
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

```
For this exercise, you require a CUDA-enabled GPU, as training an image-based model on the CPU takes a very long time.
Visit [the installation page](https://pytorch.org/get-started/locally/) to see the options available for different CUDA versions.
The remaining dependencies can be installed with pip:
```
pip install matplotlib numpy tqdm ipykernel notebook
```

## Colab Setup

Google Colab provides you with a temporary environment for python programming.
While this conveniently works on any platform and internally handles dependency issues and such, it also requires you to set up the environment from scratch every time.
The "Colab Setup" section below will be part of **every** exercise and contains utility that is needed before getting started.

**IMPORTANT**: For this exercise, you require a GPU runtime environment, as training AWAC agent on the CPU takes a very long time.
To do this, select "Change runtime type" from the context menu in the top right corner (next to the **Connect** button), and select **T4 GPU**.

There is a timeout of about ~12 hours with Colab while it is active (and less if you close your browser window).
Any changes you make to the Jupyter notebook itself should be saved to your Google Drive.
We also save all recordings and logs in it by default so that you won't lose your work in the event of an instance timeout.
However, you will need to re-mount your Google Drive and re-install packages with every new instance.

In [ ]:
#Include this at the top of your colab code
import os
if not os.path.exists('.mujoco_setup_complete'):
  # Get the prereqs
  !apt-get -qq update
  !apt-get -qq install -y libosmesa6-dev libgl1-mesa-glx libglfw3 libgl1-mesa-dev libglew-dev patchelf
  # Get Mujoco
  !mkdir ~/.mujoco
  !wget -q https://mujoco.org/download/mujoco210-linux-x86_64.tar.gz -O mujoco.tar.gz
  !tar -zxf mujoco.tar.gz -C "$HOME/.mujoco"
  !rm mujoco.tar.gz
  # Add it to the actively loaded path and the bashrc path (these only do so much)
  !echo 'export LD_LIBRARY_PATH=HOME/.mujoco/mujoco210/bin' >> ~/.bashrc
  !echo 'export LD_PRELOAD=$LD_PRELOAD:/usr/lib/x86_64-linux-gnu/libGLEW.so' >> ~/.bashrc
  # THE ANNOYING ONE, FORCE IT INTO LDCONFIG SO WE ACTUALLY GET ACCESS TO IT THIS SESSION
  !echo "/root/.mujoco/mujoco210/bin" > /etc/ld.so.conf.d/mujoco_ld_lib_path.conf
  !ldconfig
  # Install Mujoco-py
  !pip3 install -U 'mujoco-py<2.2,>=2.1'
  !pip install 'cython<3'
  # run once
  !touch .mujoco_setup_complete

try:
  if _mujoco_run_once:
    pass
except NameError:
  _mujoco_run_once = False
if not _mujoco_run_once:
  # Add it to the actively loaded path and the bashrc path (these only do so much)
  try:
    os.environ['LD_LIBRARY_PATH']=os.environ['LD_LIBRARY_PATH'] + ':/root/.mujoco/mujoco210/bin'
    os.environ['LD_LIBRARY_PATH']=os.environ['LD_LIBRARY_PATH'] + ':/usr/lib/nvidia'
  except KeyError:
    os.environ['LD_LIBRARY_PATH']='/root/.mujoco/mujoco210/bin'
  try:
    os.environ['LD_PRELOAD']=os.environ['LD_PRELOAD'] + ':/usr/lib/x86_64-linux-gnu/libGLEW.so'
  except KeyError:
    os.environ['LD_PRELOAD']='/usr/lib/x86_64-linux-gnu/libGLEW.so'
  # presetup so we don't see output on first env initialization
  # !pip install 'cython<3'
  import mujoco_py
  _mujoco_run_once = True
  print("Successfully install mujoco_py, congratulations!")

In [ ]:
%pip install -f https://download.pytorch.org/whl/torch_stable.html \
                einops\
                gym \
                protobuf\
                git+https://github.com/tinkoff-ai/d4rl@master#egg=d4rl

!pip install tqdm

# Exercise 5: Imitation Learning and Offline Reinforcement Learning

In this homework, we are going to implement **self-attention**, **behavioral cloning (BC)** and **advantage weighted actor-critic (AWAC)** algorithms

## Part 1: Self-Attention (4pts)
In this exercise, you will implement a function to compute the most fundamental variant of self-attention, known as the scaled dot-product attention. This mechanism is a cornerstone in many state-of-the-art models, particularly in natural language processing and imitation learning.

Your task is to fill in a function that computes the scaled dot-product attention of input vectors. The input to your function will be three matrices: Queries (Q), Keys (K), and Values (V), each with a shape of $(n, d_k)$, where $n$ represents the number of embeddings and $d_k$ represents the embedding dimension.

The scaled dot-product attention is mathematically represented as:
$$
\text{Attention}(Q,K,V) = \text{softmax}(\frac{Q K^T}{\sqrt{d_k}})V
$$

In [ ]:
import math
import torch
import torch.nn.functional as F
from tqdm import tqdm

torch.manual_seed(42)
torch.backends.cudnn.deterministic = True

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float32

def self_attention(Q, K, V):
    """
    Implement the self-attention mechanism.

    Parameters:
    Q (torch.Tensor): Query matrix
    K (torch.Tensor): Key matrix
    V (torch.Tensor): Value matrix

    Returns:
    torch.Tensor: Output of self-attention mechanism
    """

    # Step 1: Calculate attention scores
    # Step 2: Apply softmax to get attention weights
    # Step 3: Calculate the output as weighted sum of values

    ## TODO ##

    ## END ##

    return output


for _ in tqdm(range(2000)):
    inputs = torch.randn(10, 20).to(DEVICE).to(dtype)
    d_k = inputs.shape[-1]

    ### Initialize the weights matrix for Q, K, V
    W_q = torch.randn(d_k, d_k).to(DEVICE).to(dtype)
    W_k = torch.randn(d_k, d_k).to(DEVICE).to(dtype)
    W_v = torch.randn(d_k, d_k).to(DEVICE).to(dtype)

    ### Calculate the Q, K, V from embeddings
    Q = torch.matmul(inputs, W_q)
    K = torch.matmul(inputs, W_k)
    V = torch.matmul(inputs, W_v)

    # Call your self_attention function
    output_vectors = self_attention(Q, K, V)

    # Verify the results
    self_attention_builtin = F.scaled_dot_product_attention(Q, K, V)
    assert torch.allclose(self_attention_builtin, output_vectors, atol=1e-4),"\n The implementation seems to be incorrect!"

print("\n The implementation seems to be correct!")

## Imports and Utilities for Part 2 and 3

**Hint** :After execute the following block, there could be some warnings regarding 'flow', 'carla' and OpenGL, feel free to ignore them ☺

In [ ]:
import os
import random
from copy import deepcopy
from typing import Any, Dict, List, Optional, Tuple, Union

import d4rl
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from tqdm import trange

import matplotlib.pyplot as plt
from IPython.display import clear_output

In [ ]:
def update_plot(iterations, rewards, algorithm='AWAC'):
    """
    Update the training progress plot with new iterations and rewards.
    Designed to work in environments like Google Colab.

    Args:
    iterations (list or array): A list or array of iteration numbers.
    rewards (list or array): A list or array of rewards (normalized scores in
    this exercise) corresponding to the iterations.
    """
    clear_output(wait=True)
    plt.figure(figsize=(10, 6))
    plt.plot(iterations, rewards, label=f'{algorithm}')
    plt.xlabel('Iterations')
    plt.ylabel('Normalized Score')
    plt.title(f'Learning Curve of the {algorithm} Algorithm')
    plt.legend()
    plt.grid(True)
    plt.show()


def set_seed(
    seed: int, env: Optional[gym.Env] = None
):
    """
    Sets a fixed random seed for various modules and the environment (if provided)
    to ensure reproducibility.

    Parameters:
    - seed (int): The random seed to set.
    - env (Optional[gym.Env]): The gym environment to set the seed for.
    Defaults to None.

    Description:
    - If an environment is provided, its seed and action space seed are set to
    ensure consistent results in the environment.
    - Sets the PYTHONHASHSEED environment variable, which controls the randomness
    of Python's hash-based operations.
    - Initializes the random seeds for NumPy, Python's built-in random module,
    and PyTorch to ensure consistent random number generation across these libraries.
    """
    if env is not None:
        env.seed(seed)
        env.action_space.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)


def wrap_env(
    env: gym.Env,
    state_mean: Union[np.ndarray, float] = 0.0,
    state_std: Union[np.ndarray, float] = 1.0,
) -> gym.Env:
    """
    Wraps a gym environment to normalize its states.

    Parameters:
    - env (gym.Env): The gym environment to wrap.
    - state_mean (Union[np.ndarray, float]): The mean used for state normalization, can be a scalar or an array. Defaults to 0.0.
    - state_std (Union[np.ndarray, float]): The standard deviation used for state normalization, can be a scalar or an array. Defaults to 1.0.

    Returns:
    - gym.Env: The wrapped gym environment with state normalization.

    Description:
    - Normalizes the states of the environment by subtracting the mean and dividing by the standard deviation.
    - Utilizes the TransformObservation wrapper from gym.wrappers to apply the normalization.
    """
    def normalize_state(state):
        return (state - state_mean) / state_std

    env = gym.wrappers.TransformObservation(env, normalize_state)
    return env


@torch.no_grad()
def evaluate_policy(
        env: gym.Env, policy: torch.nn.Module, num_episodes: int = 10,
        seed: int = 0) -> np.ndarray:
    """
    Evaluates a policy in the given environment over a specified number of episodes.

    Parameters:
    - env (gym.Env): The gym environment to evaluate the policy in.
    - policy (torch.nn.Module): The policy model to evaluate.
    - num_episodes (int): The number of episodes to run the evaluation for. Defaults to 10.
    - seed (int): The random seed for environment reproducibility. Defaults to 0.

    Returns:
    - np.ndarray: An array of cumulative rewards received in each episode.

    Description:
    - Sets the seed for the environment for consistent episode generation.
    - Puts the policy in evaluation mode to disable any training-specific operations like dropout.
    - Runs the policy for a specified number of episodes, collecting total rewards per episode.
    - Switches the policy back to training mode after evaluation.
    - Uses PyTorch's no_grad context manager to disable gradient calculations, improving performance.
    """
    env.seed(seed)
    policy.eval()
    episode_rewards = []
    for _ in range(num_episodes):
        state = env.reset()
        episode_reward = 0.0
        done = False
        while not done:
            action = policy.act(state, device=DEVICE)
            state, reward, done, _ = env.step(action)
            episode_reward += reward
        episode_rewards.append(episode_reward)
    policy.train()
    return np.array(episode_rewards)

**Policy Network (Actor)**

The following neural network is trained to represent the policy `p(a|s)`, the forward function return action **sampled** from predict distribution, along with its log-probability.

In [ ]:
class Actor(nn.Module):
    def __init__(
        self,
        state_dim: int,
        action_dim: int,
        hidden_dim: int,
        min_log_std: float = -20.0,
        max_log_std: float = 2.0,
        min_action: float = -1.0,
        max_action: float = 1.0,
    ):
        """
        An actor class that defines a policy network for reinforcement learning.

        Parameters:
        - state_dim (int): Dimension of the state space.
        - action_dim (int): Dimension of the action space.
        - hidden_dim (int): Dimension of the hidden layers in the neural network.
        - min_log_std (float): Minimum logarithm of the standard deviation for the action distribution. Defaults to -20.0.
        - max_log_std (float): Maximum logarithm of the standard deviation for the action distribution. Defaults to 2.0.
        - min_action (float): Minimum action value. Defaults to -1.0.
        - max_action (float): Maximum action value. Defaults to 1.0.

        Description:
        - Initializes a policy network using a multilayer perceptron (MLP) with ReLU activations.
        - The network predicts the mean of the action distribution.
        - The standard deviation of the action distribution is parameterized as a learnable parameter, clamped between specified min and max log values.
        - The action range is bounded between specified min and max values.
        """
        super().__init__()
        self._mlp = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, action_dim),
        )
        self._log_std = nn.Parameter(torch.zeros(action_dim, dtype=torch.float32))
        self._min_log_std = min_log_std
        self._max_log_std = max_log_std
        self._min_action = min_action
        self._max_action = max_action

    def _get_policy(self, state: torch.Tensor) -> torch.distributions.Distribution:
        """
        Computes the policy distribution for a given state.

        Parameters:
        - state (torch.Tensor): The state tensor.

        Returns:
        - torch.distributions.Distribution: The policy distribution as a Normal distribution.

        Description:
        - Passes the state through the MLP to get the mean of the action distribution.
        - Clamps the logarithm of the standard deviation within the specified range.
        - Constructs a Normal distribution with the computed mean and standard deviation.
        """
        mean = self._mlp(state)
        log_std = self._log_std.clamp(self._min_log_std, self._max_log_std)
        policy = torch.distributions.Normal(mean, log_std.exp())
        return policy

    def log_prob(self, state: torch.Tensor, action: torch.Tensor) -> torch.Tensor:
        """
        Computes the log probability of a given action under the policy for a given state.

        Parameters:
        - state (torch.Tensor): The state tensor.
        - action (torch.Tensor): The action tensor.

        Returns:
        - torch.Tensor: The log probability of the action under the policy.

        Description:
        - Retrieves the policy distribution for the given state.
        - Calculates the log probability of the given action under this distribution.
        - Sums the log probabilities across the action dimensions.
        """
        policy = self._get_policy(state)
        log_prob = policy.log_prob(action).sum(-1, keepdim=True)
        return log_prob

    def forward(self, state: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Defines the forward pass for the policy network.

        Parameters:
        - state (torch.Tensor): The state tensor.

        Returns:
        - Tuple[torch.Tensor, torch.Tensor]: A tuple of the action tensor and its log probability.

        Description:
        - Gets the policy distribution for the given state.
        - Samples an action from this distribution using the reparameterization trick.
        - Clamps the action within the specified range.
        - Computes the log probability of the sampled action.
        """
        policy = self._get_policy(state)
        action = policy.rsample()
        action.clamp_(self._min_action, self._max_action)
        log_prob = policy.log_prob(action).sum(-1, keepdim=True)
        return action, log_prob

    def act(self, state: np.ndarray, device: str) -> np.ndarray:
        """
        Determines the action to take for a given state.

        Parameters:
        - state (np.ndarray): The state array.
        - device (str): The device to perform computations on.

        Returns:
        - np.ndarray: The action array.

        Description:
        - Converts the state to a PyTorch tensor and sends it to the specified device.
        - Computes the policy distribution for the given state.
        - If the network is in training mode, samples an action from the distribution; otherwise, uses the mean of the distribution.
        - Converts the action back to a NumPy array and returns it.
        """
        state_t = torch.tensor(state[None], dtype=torch.float32, device=device)
        policy = self._get_policy(state_t)
        if self._mlp.training:
            action_t = policy.sample()
        else:
            action_t = policy.mean
        action = action_t[0].cpu().numpy()
        return action

**Deep Q-Network (Critic)**

The following critic network is trained to represent the Q-function, the input is the state `s` and action `a`, and the output is Q value for this state-action pair.  

In [ ]:
class Critic(nn.Module):

    def __init__(
        self,
        state_dim: int,
        action_dim: int,
        hidden_dim: int,
    ):
        """
        Initializes the Critic model, which is a neural network used for value estimation in reinforcement learning.

        The Critic model evaluates the quality of actions taken in a given state. It is used to guide the training
        of the Actor model by providing feedback on the expected returns of the actions it selects.

        Parameters:
        - state_dim (int): The dimensionality of the state space.
        - action_dim (int): The dimensionality of the action space.
        - hidden_dim (int): The number of neurons in each hidden layer.

        The Critic network takes both state and action as inputs and outputs a Q-value representing the expected return
        of taking the given action in the given state. The network consists of a Multi-Layer Perceptron (MLP) with ReLU
        activations.
        """
        super().__init__()
        self._mlp = nn.Sequential(
            nn.Linear(state_dim + action_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1),
        )

    def forward(self, state: torch.Tensor, action: torch.Tensor) -> torch.Tensor:
        """
        Defines the forward pass through the Critic network.

        Parameters:
        - state (torch.Tensor): The state tensor for which the Q-value is to be evaluated.
        - action (torch.Tensor): The action tensor for which the Q-value is to be evaluated.

        Returns:
        - torch.Tensor: The Q-value of the given state-action pair.

        This method concatenates the state and action tensors and passes them through the MLP to output
        a Q-value. This value represents the Critic's estimate of the expected return for taking the given
        action in the given state.
        """
        q_value = self._mlp(torch.cat([state, action], dim=-1))
        return q_value

**Replay Buffer**

The Replay Buffer stores transitions (state, action, reward, next state, done). These stored transitions are used to train the agent by sampling random mini-batches, enabling the agent to learn from past experiences (replay). Different from the online replay buffer that used in online RL, in this exercise, we will use a fixed offline replay buffer that loaded from an offline collected dataset.

In [ ]:
class ReplayBuffer:

    def __init__(
        self,
        state_dim: int,
        action_dim: int,
        buffer_size: int,
        device: str = "cpu",
    ):
        """
        Initializes a Replay Buffer for storing experiences in reinforcement learning.

        Parameters:
        - state_dim (int): Dimensionality of the state space.
        - action_dim (int): Dimensionality of the action space.
        - buffer_size (int): The maximum number of transitions the buffer can hold.
        - device (str): The device on which the tensors are stored. Default is 'cpu'.

        The buffer initializes tensors to store states, actions, rewards, next states, and done flags.
        Each tensor has a size determined by the buffer_size and the respective dimensionality of its content.
        """
        self._buffer_size = buffer_size
        self._pointer = 0
        self._size = 0

        self._states = torch.zeros(
            (buffer_size, state_dim), dtype=torch.float32, device=device
        )
        self._actions = torch.zeros(
            (buffer_size, action_dim), dtype=torch.float32, device=device
        )
        self._rewards = torch.zeros((buffer_size, 1), dtype=torch.float32, device=device)
        self._next_states = torch.zeros(
            (buffer_size, state_dim), dtype=torch.float32, device=device
        )
        self._dones = torch.zeros((buffer_size, 1), dtype=torch.float32, device=device)
        self._device = device

    def _to_tensor(self, data: np.ndarray) -> torch.Tensor:
        """
        Converts numpy array data to a PyTorch tensor.

        Parameters:
        - data (np.ndarray): Data to be converted.

        Returns:
        - torch.Tensor: The corresponding tensor on the specified device.

        This method is used to convert numpy arrays to PyTorch tensors, ensuring that all data in the
        buffer is in tensor format for efficient processing and batched learning.
        """
        return torch.tensor(data, dtype=torch.float32, device=self._device)

    def load_d4rl_dataset(self, data: Dict[str, np.ndarray]):
        """
        Loads a dataset from D4RL (Datasets for Deep Data-Driven Reinforcement Learning) into the buffer.

        Parameters:
        - data (Dict[str, np.ndarray]): The dataset to be loaded, typically containing 'observations',
          'actions', 'rewards', 'next_observations', and 'terminals'.

        This method loads the entire dataset into the buffer, assuming the buffer is empty and large enough
        to hold the dataset. It raises an error if the buffer is non-empty or smaller than the dataset.
        """
        if self._size != 0:
            raise ValueError("Trying to load data into non-empty replay buffer")
        n_transitions = data["observations"].shape[0]
        if n_transitions > self._buffer_size:
            raise ValueError(
                "Replay buffer is smaller than the dataset you are trying to load!"
            )
        self._states[:n_transitions] = self._to_tensor(data["observations"])
        self._actions[:n_transitions] = self._to_tensor(data["actions"])
        self._rewards[:n_transitions] = self._to_tensor(data["rewards"][..., None])
        self._next_states[:n_transitions] = self._to_tensor(data["next_observations"])
        self._dones[:n_transitions] = self._to_tensor(data["terminals"][..., None])
        self._size += n_transitions
        self._pointer = min(self._size, n_transitions)

        print(f"Dataset size: {n_transitions}")

    def sample(self, batch_size: int) -> List[torch.Tensor]:
        """
        Samples a mini-batch of experiences from the buffer.

        Parameters:
        - batch_size (int): The size of the mini-batch to sample.

        Returns:
        - List[torch.Tensor]: A list containing tensors of states, actions, rewards, next states, and dones.

        Randomly samples a batch of experiences from the buffer. This method is used during the training
        process to provide the agent with a diverse set of experiences from which to learn, breaking the
        correlation present in sequential experiences.
        """
        indices = np.random.randint(0, min(self._size, self._pointer), size=batch_size)
        states = self._states[indices]
        actions = self._actions[indices]
        rewards = self._rewards[indices]
        next_states = self._next_states[indices]
        dones = self._dones[indices]
        return [states, actions, rewards, next_states, dones]

**Setup environmental hyperparameters**

In [ ]:
ENV_NAME = "halfcheetah-medium-expert-v2"
SEED = 42
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

## Part 2: Behavioral Cloning (BC) (3pts)
**Behavioral Cloning (BC)** is a classic imitation learning method where a model, typically a neural network, is trained to "cloning" the behavioral of an expert from expert demonstrations.

1. **Data Collection:**
   - Collect a dataset $\mathcal{D}$ of state-action pairs $\{(s_i, a_i)\}$ from expert demonstrations.

2. **Initialize:**
   - Initialize the policy network $\pi_\theta(a|s)$ with parameters $\theta$.

3. **Training Loop:**
   - **For each epoch** or until convergence:
     - Shuffle the dataset $\mathcal{D}$.
     - **For each mini-batch** $\mathcal{B}$ in $\mathcal{D}$:
       - Extract mini-batch of state-action pairs $(s, a)$.
       - Compute the log-likelihood loss: $$ L(\theta) = -\frac{1}{|\mathcal{B}|} \sum_{(s, a) \in B} \log \pi_\theta(a|s) $$
       - Update the parameters $\theta$ of the policy network using gradient descent to minimize the loss.

Tip:
- There are multiple choices of the loss function in behavioral cloning, e.g., Mean Square Error (MSE) and Cross Entropy (CE). In this exercise, we focus on maximizing the log-likelihood as it is more straightforward to compare with AWAC.

In [ ]:
class BehavioralCloning:
    """
    A Behavioral Cloning class for training an agent using expert demonstrations.

    Attributes:
        _actor (nn.Module): The policy network that outputs actions given states.
        _actor_optimizer (torch.optim.Optimizer): Optimizer for training the actor.
        _device (str): The device (e.g., 'cpu' or 'cuda') on which to perform computations.
    """
    def __init__(self,
                 actor: nn.Module,
                 actor_optimizer: torch.optim.Optimizer,
                 device: str = "cpu"):
        """
        Initializes the BehavioralCloning class with an actor network, its optimizer, and the device.

        Args:
            actor (nn.Module): The policy network.
            actor_optimizer (torch.optim.Optimizer): Optimizer for the actor.
            device (str, optional): Computation device, 'cpu' by default.
        """
        self._actor = actor
        self._actor_optimizer = actor_optimizer
        self._device = device

    def _actor_loss(self, states: torch.Tensor, actions: torch.Tensor) -> torch.Tensor:
        """
        Computes the loss for the actor based on the log likelihood of the actions given the states.

        Args:
            states (torch.Tensor): The input states.
            actions (torch.Tensor): The expert actions corresponding to the states.

        Returns:
            torch.Tensor: The computed loss for the actor.
        """
        ## TODO ##

        ## END ##
        return loss

    def _update_actor(self, states: torch.Tensor, actions: torch.Tensor) -> torch.Tensor:
        """
        Performs a single optimization step for the actor.

        Args:
            states (torch.Tensor): States from the replay buffer.
            actions (torch.Tensor): Corresponding actions from the replay buffer.

        Returns:
            float: The loss value after the optimization step.
        """
        self._actor_optimizer.zero_grad()
        loss = self._actor_loss(states, actions)
        loss.backward()
        self._actor_optimizer.step()
        return loss.item()

    def update(self,
              replay_buffer: ReplayBuffer,
              batch_size: int) -> torch.Tensor:
        """
        Updates the actor using a batch of data from the replay buffer.

        Args:
            replay_buffer (ReplayBuffer): The replay buffer to sample experiences from.
            batch_size (int): The number of samples to draw from the replay buffer.

        Returns:
            float: The loss value from the actor update.
        """
        states, actions, _, _, _ = replay_buffer.sample(batch_size)
        actor_loss = self._update_actor(states, actions)
        return actor_loss

In [ ]:
def bc_train():
    env = gym.make(ENV_NAME)

    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.shape[0]

    dataset = d4rl.qlearning_dataset(env)

    state_mean = dataset['observations'].mean(axis=0)
    state_std = dataset['observations'].std(axis=0)

    # Normalize the observations in the dataset
    dataset['observations'] = (dataset['observations'] - state_mean) / state_std

    # Wrap the environment to apply state normalization during training
    env = wrap_env(env, state_mean=state_mean, state_std=state_std)

    replay_buffer = ReplayBuffer(
        state_dim=state_dim,
        action_dim=action_dim,
        buffer_size=2_000_000,
        device=DEVICE,
    )

    replay_buffer.load_d4rl_dataset(dataset)
    set_seed(SEED, env)

    actor_kwargs = {
        "state_dim": state_dim,
        "action_dim": action_dim,
        "hidden_dim": 256,
    }

    actor = Actor(**actor_kwargs)
    actor.to(DEVICE)
    actor_optimizer = torch.optim.Adam(actor.parameters(), lr=3e-4)

    bc = BehavioralCloning(actor=actor,
                           actor_optimizer=actor_optimizer,
                           device=DEVICE)

    eval_score_list = [ ]
    iteration_list = [ ]
    for t in trange(60000, ncols=100):
        actor_loss = bc.update(replay_buffer, batch_size=256)
        if (t + 1) % 1000 == 0:
            eval_scores = evaluate_policy(env, actor, num_episodes=10)
            normalized_eval_scores = env.get_normalized_score(eval_scores)
            print(f"Eval_scores: {eval_scores.mean():.2f} +/- {eval_scores.std():.2f}")
            print(f"Normalized eval_scores: {normalized_eval_scores.mean():.2f} +/- {normalized_eval_scores.std():.2f}")
            print(f"Actor loss: {actor_loss:.4f}")
            eval_score_list.append(normalized_eval_scores.mean())
            iteration_list.append(t+1)
            update_plot(iteration_list, eval_score_list, 'BC')

**Run the following block to train your BC agent!**

Hint:
- the BC agent with default hyperparameters should be able to achieve normalized scores >= 0.5

In [ ]:
bc_train()

## Part 3: Advantage-Weighted Actor-Critic (AWAC) (8pts)
In this part, we will implement the AWAC algorithm with double critic, which is similar to the technique used in Soft Actor-Critic(SAC) and TD3. The algorithm looks as follow. The AWAC algorithm can be used both as online or offline algorithm, in this exercise we focus only on its offline variant.

1. **Initialize:**
   - Initialize actor network $\pi_\theta(a|s)$ with parameters $\theta$.
   - Initialize two critic networks $Q_{\phi_1}(s, a)$ and $Q_{\phi_2}(s, a)$ with parameters $\phi_1$ and $\phi_2$.
   - Initialize target critic networks $Q_{\phi'_1}$ and $Q_{\phi'_2}$ with parameters $\phi'_1 \leftarrow \phi_1$ and $\phi'_2 \leftarrow \phi_2$.
   - Initialize replay buffer $\mathcal{D}$.

2. **For each iteration:**
   - **For each gradient step:**
     - Sample a batch of transitions $(s, a, r, s', d)$ from $\mathcal{D}$.
     - Compute target values using the smaller Q value of the two target critics: $$ y = r + \gamma (1 - d) \min_{i=1,2} Q_{\phi'_i}(s', a') $$
     - Update each critic by minimizing the loss: $$ L(\phi_i) = \frac{1}{|B|} \sum_{(s, a, r, s', d) \in B} (Q_{\phi_i}(s, a) - y)^2 \quad \text{for } i=1,2 $$
     - Compute the advantage for a given state action pair (s, a):
     \begin{equation}
     A(s,a)=\min_{i=1,2}Q_{\phi_i}(s,a) - \mathbb{E}_{a^\pi \sim \pi_{\theta}(a|s)}[\min_{i=1,2}Q_{\phi_i}(s,a^\pi)]
     \end{equation}
     
     **Important Hint** : here we use the expectation of Q function to represent the $V(s)$. In theory, we need multiple samples to get better estimation, however, only **one sample** usually works fine in practice.
     - Compute advantage-weighted policy gradient: $$ \nabla_\theta J(\theta) = \frac{1}{|B|} \sum_{(s, a) \in B} \nabla_\theta \log \pi_\theta(a|s) \exp \left( \frac{1}{\lambda} A(s,a) \right)$$
     - Update actor network by gradient ascent: $$ \theta \leftarrow \theta + \alpha \nabla_\theta J(\theta) $$
     - Softly update target critic networks: $$ \phi'_i \leftarrow \tau \phi_i + (1 - \tau) \phi'_i \quad \text{for } i=1,2 $$

3. **Repeat** until convergence or maximum number of iterations reached.



In [ ]:
class AdvantageWeightActorCritic:
    def __init__(
        self,
        actor: nn.Module,
        actor_optimizer: torch.optim.Optimizer,
        critic_1: nn.Module,
        critic_1_optimizer: torch.optim.Optimizer,
        critic_2: nn.Module,
        critic_2_optimizer: torch.optim.Optimizer,
        gamma: float = 0.99,
        tau: float = 0.005,
        awac_lambda: float = 1.0,
        exp_adv_max: float = 100.0,
        device: str = "cpu",
    ):
        """
        Initializes the Advantage Weighted Actor-Critic (AWAC) agent.

        AWAC combines off-policy actor-critic methods with an advantage-weighted behavioral cloning loss to stabilize
        and improve the training of deep reinforcement learning policies.

        Parameters:
        - actor (nn.Module): The actor network.
        - actor_optimizer (torch.optim.Optimizer): Optimizer for the actor network.
        - critic_1 (nn.Module): The first critic network.
        - critic_1_optimizer (torch.optim.Optimizer): Optimizer for the first critic network.
        - critic_2 (nn.Module): The second critic network.
        - critic_2_optimizer (torch.optim.Optimizer): Optimizer for the second critic network.
        - gamma (float): Discount factor for future rewards. Default is 0.99.
        - tau (float): Soft update rate for the target networks. Default is 0.005.
        - awac_lambda (float): Scaling factor for advantage weights in the actor loss. Default is 1.0.
        - exp_adv_max (float): Maximum limit for exponentiated advantages. Default is 100.0.
        - device (str): The device on which to perform computations. Default is 'cpu'.

        This implementation uses two critic networks to estimate the value function, along with their target networks,
        which are updated using a soft update strategy. The actor's policy is updated using an advantage-weighted
        loss to encourage actions that lead to higher returns than currently estimated.
        """
        self._actor = actor
        self._actor_optimizer = actor_optimizer

        self._critic_1 = critic_1
        self._critic_1_optimizer = critic_1_optimizer
        self._target_critic_1 = deepcopy(critic_1)

        self._critic_2 = critic_2
        self._critic_2_optimizer = critic_2_optimizer
        self._target_critic_2 = deepcopy(critic_2)

        self._gamma = gamma
        self._tau = tau
        self._awac_lambda = awac_lambda
        self._exp_adv_max = exp_adv_max

        self._device = device

    def _actor_loss(self, states: torch.Tensor, actions: torch.Tensor) -> torch.Tensor:
        """
        Computes the loss for the actor network.

        The loss is calculated using an advantage-weighted behavioral cloning approach. It encourages the actor to
        take actions that lead to higher returns compared to the current policy's estimate.

        Parameters:
        - states (torch.Tensor): The batch of states.
        - actions (torch.Tensor): The batch of actions.

        Returns:
        - torch.Tensor: The computed loss for the actor network.

        The advantage is calculated as the difference between the Q-values from the critics for the actual
        actions and the estimated V-values from the critics for the actions sampled by the actor. These advantages
        are then scaled and used to weight the log probabilities of the actions in the actor's policy.
        """
        with torch.no_grad():
            pi_actions, _ = self._actor(states)

            """
            Calculate advantages and the weights
              Hints:
                1. V(s) can be represented using expectation of Q(s,a).
                   In practice, one sample could be enough
                2. self._actor() return action samples from the actor
                3. Weights should not contain gradient
            """

            ## TODO ##

            ## END ##

            weights = torch.clamp(weights, max=self._exp_adv_max)

        ### Implement the AWAC loss ###
        """
        Hint: be careful about the sign of loss, we are trying to maximize the
        objective.
        """
        ## TODO ##

        ## END ##
        return loss

    def _critic_loss(self, states, actions, rewards, next_states, dones):
        """
        Computes the loss for the critic networks.

        The loss is the Mean Squared Error (MSE) between the critics' Q-value estimates and the target Q-values,
        which are computed using the Bellman equation.

        Parameters:
        - states, actions, rewards, next_states, dones: Tensors representing the batch of transitions.

        Returns:
        - torch.Tensor: The computed loss for the critic networks.

        The target Q-values are calculated using the minimum Q-value of the two target critics for the next state
        and action pair, adjusted by the reward and discount factor.
        """
        """
        Hints:
          1. next_action could be compute by calling self._actor(next_states)
          2. targets should not contain gradient
          3. we have two critics, so we need to calculate the loss for both and add them up
        """

        # Step 1: Compute the next actions

        # Step 2: Calculate the targets

        # Step 3: Compute the critic loss with targets


        ## TODO ##

        ## END ##

        loss = critic_1_loss + critic_2_loss
        return loss

    def _update_target_network(self, target_network: nn.Module, network: nn.Module):
        """
        Performs a soft update on the target network.

        Parameters:
        - target_network (nn.Module): The target network to be updated.
        - network (nn.Module): The current network whose parameters are used for the update.

        This method updates the target network's parameters by blending them with the parameters from the
        current network, controlled by the tau parameter. This soft update helps in stabilizing training by
        providing a slowly changing target for the critics.
        """
        for target_param, param in zip(target_network.parameters(), network.parameters()):
            target_param.data.copy_(self._tau * param.data + (1.0 - self._tau) * target_param.data)

    def _update_actor(self, states: torch.Tensor, actions: torch.Tensor) -> torch.Tensor:
        """
        Updates the actor network by minimizing its loss.

        Parameters:
        - states (torch.Tensor): The batch of states.
        - actions (torch.Tensor): The batch of actions.

        Returns:
        - torch.Tensor: The actor loss after the update.

        This method performs a gradient descent step to update the actor's parameters in order to minimize
        the advantage-weighted loss.
        """
        self._actor_optimizer.zero_grad()
        loss = self._actor_loss(states, actions)
        loss.backward()
        self._actor_optimizer.step()
        return loss.item()

    def _update_critics(
        self,
        states: torch.Tensor,
        actions: torch.Tensor,
        rewards: torch.Tensor,
        next_states: torch.Tensor,
        dones: torch.Tensor,
    ) -> torch.Tensor:
        """
        Updates the critic networks by minimizing their loss.

        Parameters:
        - states, actions, rewards, next_states, dones: Tensors representing the batch of transitions.

        Returns:
        - torch.Tensor: The critic loss after the update.

        This method performs a gradient descent step to update the critic networks' parameters in order to minimize
        the MSE loss between their Q-value estimates and the target Q-values.
        """
        self._critic_1_optimizer.zero_grad()
        self._critic_2_optimizer.zero_grad()
        loss = self._critic_loss(states, actions, rewards, next_states, dones)
        loss.backward()
        self._critic_1_optimizer.step()
        self._critic_2_optimizer.step()
        return loss.item()

    def update(
        self,
        replay_buffer: ReplayBuffer,
        batch_size: int,
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Performs a training update on both the actor and critic networks.

        Parameters:
        - replay_buffer (ReplayBuffer): The replay buffer to sample transitions from.
        - batch_size (int): The number of transitions to sample for the update.

        Returns:
        - Tuple[torch.Tensor, torch.Tensor]: The actor and critic losses after the update.

        This method samples a batch of transitions from the replay buffer and uses them to update both the actor
        and critic networks. It also updates the target critic networks using the soft update method.
        """
        states, actions, rewards, next_states, dones = replay_buffer.sample(batch_size)
        critic_loss = self._update_critics(states, actions, rewards, next_states, dones)
        actor_loss = self._update_actor(states, actions)
        self._update_target_network(self._target_critic_1, self._critic_1)
        self._update_target_network(self._target_critic_2, self._critic_2)
        return actor_loss, critic_loss

In [ ]:
def awac_train():

    # Environment setup and seeding
    env = gym.make(ENV_NAME)
    set_seed(SEED, env)
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.shape[0]
    dataset = d4rl.qlearning_dataset(env)

    state_mean = dataset['observations'].mean(axis=0)
    state_std = dataset['observations'].std(axis=0)

    dataset['observations'] = (dataset['observations'] - state_mean) / state_std
    dataset['next_observations'] = (dataset['next_observations'] - state_mean) / state_std

    env = wrap_env(env, state_mean=state_mean, state_std=state_std)

    # Initialize replay buffer
    replay_buffer = ReplayBuffer(
        state_dim=state_dim,
        action_dim=action_dim,
        buffer_size=2_000_000,
        device=DEVICE,
    )
    replay_buffer.load_d4rl_dataset(dataset)

    actor_critic_kwargs = {
        "state_dim": state_dim,
        "action_dim": action_dim,
        "hidden_dim": 256,
    }

    actor = Actor(**actor_critic_kwargs)
    actor.to(DEVICE)
    actor_optimizer = torch.optim.Adam(actor.parameters(), lr=3e-4)

    critic_1 = Critic(**actor_critic_kwargs)
    critic_1.to(DEVICE)
    critic_1_optimizer = torch.optim.Adam(critic_1.parameters(), lr=3e-4)

    critic_2 = Critic(**actor_critic_kwargs)
    critic_2.to(DEVICE)
    critic_2_optimizer = torch.optim.Adam(critic_2.parameters(), lr=3e-4)

    awac = AdvantageWeightActorCritic(
        actor=actor,
        actor_optimizer=actor_optimizer,
        critic_1=critic_1,
        critic_1_optimizer=critic_1_optimizer,
        critic_2=critic_2,
        critic_2_optimizer=critic_2_optimizer,
        gamma=0.99,
        tau=0.005,
        awac_lambda=1.0,
        exp_adv_max=100.0,
        device=DEVICE,
    )

    eval_score_list = []
    iteration_list = []
    for t in trange(60000, ncols=100):
        actor_loss, critic_loss = awac.update(replay_buffer, batch_size=1024)
        if (t + 1) % 1000 == 0:
            eval_scores = evaluate_policy(env, actor, num_episodes=10)
            normalized_eval_scores = env.get_normalized_score(eval_scores)
            print(f"Eval_scores: {eval_scores.mean():.2f} +/- {eval_scores.std():.2f}")
            print(f"Normalized eval_scores: {normalized_eval_scores.mean():.2f} +/- {normalized_eval_scores.std():.2f}")
            print(f"Actor loss: {actor_loss:.4f}, critic loss: {critic_loss:.4f}")
            eval_score_list.append(normalized_eval_scores.mean())
            iteration_list.append(t+1)
            update_plot(iteration_list, eval_score_list, 'AWAC')

**Run the following block to train the awac agent!**

Hints:
- the AWAC agent with default hyperparameters should be able to achieve normalized scores >= 0.8
- There could be a drop of performance at the beginning of training. You should be able to observe significant performance improvement after 20000 iterations.
- It takes 10~12 minutes to finish training on T4-GPU runtime.

In [ ]:
awac_train()

## Self-test questions (optional)
Why AWAC can achieve better performance than BC?

.## TODO ##

What are the challenges of applying offline RL in the real world? （Open-ended)

.## TODO ##